In [15]:
import pandas as pd
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
import numpy as np
import os
import time
from tqdm import tqdm
import matplotlib.pyplot as plt
import seaborn as sns
import datetime
tqdm.pandas()

In [2]:
os.listdir('../국토교통부_실거래가_공개시스템/')

['경기도',
 '단독다가구',
 '땅값분석',
 '땅집값 비교분석',
 '땅집값 비교분석.zip',
 '땅집값 비교분석2',
 '땅집값 비교분석2.zip',
 '땅집값 비교분석3',
 '땅집값 비교분석4',
 '땅집값_비교분석_최근12개월_구별',
 '땅집값_비교분석_최근12개월_구별.zip',
 '땅집값_비교분석_최근6개월_구별',
 '땅집값_비교분석_최근6개월_구별.zip',
 '분양입주권',
 '비교분석',
 '상업업무용',
 '실거래가_다운로드_진행상황.txt',
 '아파트',
 '연립다세대',
 '오피스텔',
 '집값분석',
 '토지']

In [3]:
basedir = '../국토교통부_실거래가_공개시스템/'

In [4]:
apart_rent_basedir = basedir + '아파트/전월세/'
apart_trade_basedir = basedir + '아파트/매매/'

yunrip_rent_basedir = basedir + '연립다세대/전월세/'
yunrip_trade_basedir = basedir + '연립다세대/매매/'

officetel_rent_basedir = basedir + '오피스텔/전월세/'
officetel_trade_basedir = basedir + '오피스텔/매매/'

In [5]:
def data_prep(basedir):

    filenames = [f for f in os.listdir(basedir) if (f.endswith('.csv'))]
    
    cols_to_drop = ['계약기간', '계약구분', '갱신요구권 사용']
    
    dfs_list = []
    for file in tqdm(filenames):
        df = pd.read_csv(basedir + file, encoding='euc-kr', header=15)
        
        if df.shape[0] == 0:
            continue
            
        if '해제사유발생일' in df.columns:
            df = df[df['해제사유발생일'].isna()]
            df = df.drop(columns=['해제사유발생일'])
        
        if '중개사소재지' in df.columns:
            df = df.drop(columns=['중개사소재지'])
            
        if '대지권면적(㎡)' in df.columns:
            df = df.drop(columns=['대지권면적(㎡)'])
            
        df = df.drop(columns=[col for col in df.columns if '종전' in col] + [col for col in df.columns if col in cols_to_drop])
        
        dfs_list.append(df)
    
    concat_df = pd.concat(dfs_list).reset_index(drop=True)
    
    for col in concat_df.columns:
        if '(만원)' in col:
            concat_df[col] = concat_df[col].apply(lambda x: int(str(x).replace(',', '')))
            
    if '보증금(만원)' in concat_df.columns:
        concat_df = concat_df.rename(columns={'보증금(만원)':'거래/보증금(만원)'})
    
    if '거래금액(만원)' in concat_df.columns:
        concat_df = concat_df.rename(columns={'거래금액(만원)':'거래/보증금(만원)'})
        
    if '건물명' in concat_df.columns:
        concat_df = concat_df.rename(columns={'건물명':'건물/단지명'})
        
    if '단지명' in concat_df.columns:
        concat_df = concat_df.rename(columns={'단지명':'건물/단지명'})
        
    
    return concat_df

In [6]:
apart_rent_df = data_prep(apart_rent_basedir)
apart_rent_df['부동산유형'] = '아파트'
apart_rent_df = apart_rent_df.rename(columns={'전월세구분':'거래유형'})
print(apart_rent_df.shape)
apart_rent_df.head()

  0%|                                                                                           | 0/11 [00:00<?, ?it/s]c:\users\max6296\ml\lib\site-packages\IPython\core\interactiveshell.py:3338: DtypeWarning: Columns (10) have mixed types.Specify dtype option on import or set low_memory=False.
  if (await self.run_code(code, result,  async_=asy)):
100%|██████████████████████████████████████████████████████████████████████████████████| 11/11 [00:03<00:00,  3.48it/s]


(1792351, 15)


,시군구,번지,본번,부번,건물/단지명,거래유형,전용면적(㎡),계약년월,계약일,거래/보증금(만원),월세(만원),층,건축년도,도로명,부동산유형
0,서울특별시 강남구 개포동,655-2,655.0,2.0,개포2차현대아파트(220),전세,77.75,201101,5,35000,0,7.0,1988.0,언주로 103,아파트
1,서울특별시 강남구 개포동,655-2,655.0,2.0,개포2차현대아파트(220),전세,77.75,201101,18,20000,0,8.0,1988.0,언주로 103,아파트
2,서울특별시 강남구 개포동,655-2,655.0,2.0,개포2차현대아파트(220),전세,77.75,201102,1,24000,0,5.0,1988.0,언주로 103,아파트
3,서울특별시 강남구 개포동,655-2,655.0,2.0,개포2차현대아파트(220),전세,77.75,201102,11,31000,0,9.0,1988.0,언주로 103,아파트
4,서울특별시 강남구 개포동,655-2,655.0,2.0,개포2차현대아파트(220),전세,77.75,201102,24,30500,0,9.0,1988.0,언주로 103,아파트


In [7]:
apart_trade_df = data_prep(apart_trade_basedir)
apart_trade_df['부동산유형'] = '아파트'
apart_trade_df['거래유형'] = '매매'
apart_trade_df['월세(만원)'] = 0
print(apart_trade_df.shape)
apart_trade_df.head()

100%|██████████████████████████████████████████████████████████████████████████████████| 16/16 [00:02<00:00,  5.93it/s]


(1218608, 15)


,시군구,번지,본번,부번,건물/단지명,전용면적(㎡),계약년월,계약일,거래/보증금(만원),층,건축년도,도로명,거래유형,부동산유형,월세(만원)
0,서울특별시 강남구 개포동,655-2,655.0,2.0,개포2차현대아파트(220),77.75,200603,10,59500,7,1988,언주로 103,매매,아파트,0
1,서울특별시 강남구 개포동,655-2,655.0,2.0,개포2차현대아파트(220),77.75,200603,29,60000,6,1988,언주로 103,매매,아파트,0
2,서울특별시 강남구 개포동,655-2,655.0,2.0,개포2차현대아파트(220),77.75,200604,29,67000,9,1988,언주로 103,매매,아파트,0
3,서울특별시 강남구 개포동,655-2,655.0,2.0,개포2차현대아파트(220),77.75,200606,1,60000,4,1988,언주로 103,매매,아파트,0
4,서울특별시 강남구 개포동,655-2,655.0,2.0,개포2차현대아파트(220),77.75,200610,20,72250,5,1988,언주로 103,매매,아파트,0


In [8]:
yunrip_rent_df = data_prep(yunrip_rent_basedir)
yunrip_rent_df['부동산유형'] = '연립다세대'
yunrip_rent_df = yunrip_rent_df.rename(columns={'전월세구분':'거래유형'})
print(yunrip_rent_df.shape)
yunrip_rent_df.head()

 55%|█████████████████████████████████████████████▎                                     | 6/11 [00:01<00:00,  5.63it/s]c:\users\max6296\ml\lib\site-packages\IPython\core\interactiveshell.py:3338: DtypeWarning: Columns (10) have mixed types.Specify dtype option on import or set low_memory=False.
  if (await self.run_code(code, result,  async_=asy)):
100%|██████████████████████████████████████████████████████████████████████████████████| 11/11 [00:02<00:00,  5.32it/s]


(929546, 15)


,시군구,번지,본번,부번,건물/단지명,거래유형,전용면적(㎡),계약년월,계약일,거래/보증금(만원),월세(만원),층,건축년도,도로명,부동산유형
0,서울특별시 강남구 개포동,1252-8,1252,8,(1252-8),전세,65.92,201103,26,20000,0,4.0,2006.0,개포로21길 21,연립다세대
1,서울특별시 강남구 개포동,1252-8,1252,8,(1252-8),월세,65.92,201110,13,15000,50,5.0,2006.0,개포로21길 21,연립다세대
2,서울특별시 강남구 개포동,1265-10,1265,10,(1265-10),전세,81.12,201102,15,23000,0,4.0,2006.0,개포로31길 23-10,연립다세대
3,서울특별시 강남구 개포동,159-2,159,2,(159-2),전세,57.55,201110,11,15000,0,2.0,1987.0,선릉로8길 12,연립다세대
4,서울특별시 강남구 개포동,162-6,162,6,(162-6),전세,63.63,201103,19,14000,0,2.0,1987.0,선릉로10길 22-7,연립다세대


In [9]:
yunrip_trade_df = data_prep(yunrip_trade_basedir)
yunrip_trade_df['부동산유형'] = '연립다세대'
yunrip_trade_df['거래유형'] = '매매'
yunrip_trade_df['월세(만원)'] = 0
print(yunrip_trade_df.shape)
yunrip_trade_df.head()

100%|██████████████████████████████████████████████████████████████████████████████████| 16/16 [00:02<00:00,  7.91it/s]


(723258, 15)


,시군구,번지,본번,부번,건물/단지명,전용면적(㎡),계약년월,계약일,거래/보증금(만원),층,건축년도,도로명,거래유형,부동산유형,월세(만원)
0,서울특별시 강남구 개포동,1264-3,1264,3,(1264-3),53.28,200608,29,11500,-1,1992.0,개포로31길 23-7,매매,연립다세대,0
1,서울특별시 강남구 개포동,171-13,171,13,(171-13),68.08,200612,20,56500,2,1988.0,선릉로14길 11,매매,연립다세대,0
2,서울특별시 강남구 개포동,1239-7,1239,7,강남빌라가동,52.59,200612,9,40500,2,1988.0,개포로15길 25,매매,연립다세대,0
3,서울특별시 강남구 개포동,1239-7,1239,7,강남빌라가동,52.59,200612,19,42000,1,1988.0,개포로15길 25,매매,연립다세대,0
4,서울특별시 강남구 개포동,1239-6,1239,6,강남빌라나동,52.75,200611,2,22000,2,1988.0,개포로15길 27,매매,연립다세대,0


In [10]:
officetel_rent_df = data_prep(officetel_rent_basedir)
officetel_rent_df['부동산유형'] = '오피스텔'
officetel_rent_df = officetel_rent_df.rename(columns={'전월세구분':'거래유형'})
print(officetel_rent_df.shape)
officetel_rent_df.head()

 55%|█████████████████████████████████████████████▎                                     | 6/11 [00:00<00:00, 20.74it/s]c:\users\max6296\ml\lib\site-packages\IPython\core\interactiveshell.py:3338: DtypeWarning: Columns (10) have mixed types.Specify dtype option on import or set low_memory=False.
  if (await self.run_code(code, result,  async_=asy)):
100%|██████████████████████████████████████████████████████████████████████████████████| 11/11 [00:00<00:00, 12.98it/s]


(334289, 15)


,시군구,번지,본번,부번,건물/단지명,거래유형,전용면적(㎡),계약년월,계약일,거래/보증금(만원),월세(만원),층,건축년도,도로명,부동산유형
0,서울특별시 강남구 개포동,157-9,157,9,\t(157-9)\t,전세,29.25,201112,8,2000,0,3,2011.0,선릉로4길 19,오피스텔
1,서울특별시 강남구 개포동,1246-3,1246,3,골든빌오피스텔(1246-3),월세,23.48,201112,3,200,85,4,2003.0,개포로 213,오피스텔
2,서울특별시 강남구 개포동,13-3,13,3,대청타워,전세,32.44,201103,12,10100,0,26,1997.0,개포로 623,오피스텔
3,서울특별시 강남구 개포동,13-3,13,3,대청타워,전세,32.44,201105,23,10000,0,18,1997.0,개포로 623,오피스텔
4,서울특별시 강남구 개포동,13-3,13,3,대청타워,전세,32.44,201108,27,10000,0,20,1997.0,개포로 623,오피스텔


In [11]:
officetel_trade_df = data_prep(officetel_trade_basedir)
officetel_trade_df['부동산유형'] = '오피스텔'
officetel_trade_df['거래유형'] = '매매'
officetel_trade_df['월세(만원)'] = 0
print(officetel_trade_df.shape)
officetel_trade_df.head()

100%|██████████████████████████████████████████████████████████████████████████████████| 16/16 [00:00<00:00, 30.07it/s]

(190049, 15)


,시군구,번지,본번,부번,건물/단지명,전용면적(㎡),계약년월,계약일,거래/보증금(만원),층,건축년도,도로명,거래유형,부동산유형,월세(만원)
0,서울특별시 강남구 개포동,13-3,13,3,대청타워,32.44,200601,10,6400,8,1997.0,개포로 623,매매,오피스텔,0
1,서울특별시 강남구 개포동,13-3,13,3,대청타워,32.44,200601,10,7100,7,1997.0,개포로 623,매매,오피스텔,0
2,서울특별시 강남구 개포동,13-3,13,3,대청타워,32.44,200601,11,7300,11,1997.0,개포로 623,매매,오피스텔,0
3,서울특별시 강남구 개포동,13-3,13,3,대청타워,31.91,200601,18,7500,10,1997.0,개포로 623,매매,오피스텔,0
4,서울특별시 강남구 개포동,13-3,13,3,대청타워,32.44,200601,19,7400,15,1997.0,개포로 623,매매,오피스텔,0


In [12]:
dfs_list = [apart_rent_df, apart_trade_df, yunrip_rent_df, yunrip_trade_df, officetel_rent_df, officetel_trade_df]
concat_df = pd.concat(dfs_list).reset_index(drop=True)
print(concat_df.shape)
concat_df.head()

(5188101, 15)


,시군구,번지,본번,부번,건물/단지명,거래유형,전용면적(㎡),계약년월,계약일,거래/보증금(만원),월세(만원),층,건축년도,도로명,부동산유형
0,서울특별시 강남구 개포동,655-2,655.0,2.0,개포2차현대아파트(220),전세,77.75,201101,5,35000,0,7.0,1988.0,언주로 103,아파트
1,서울특별시 강남구 개포동,655-2,655.0,2.0,개포2차현대아파트(220),전세,77.75,201101,18,20000,0,8.0,1988.0,언주로 103,아파트
2,서울특별시 강남구 개포동,655-2,655.0,2.0,개포2차현대아파트(220),전세,77.75,201102,1,24000,0,5.0,1988.0,언주로 103,아파트
3,서울특별시 강남구 개포동,655-2,655.0,2.0,개포2차현대아파트(220),전세,77.75,201102,11,31000,0,9.0,1988.0,언주로 103,아파트
4,서울특별시 강남구 개포동,655-2,655.0,2.0,개포2차현대아파트(220),전세,77.75,201102,24,30500,0,9.0,1988.0,언주로 103,아파트


In [13]:
concat_df['건물/단지명'] = concat_df['건물/단지명'].fillna('없음')

In [14]:
def price_range(x):
    # x is a number
    
    if pd.isna(x) == True:
        return x
    else:
        
        if x <= 14000:
            return '1억4천 이하'
        
        for i in range(16000, 92000, 2000):
            if x <= i:
                return '{}~{}'.format(i-2000, i)
        
        if x > 90000:
            return '9억이상'

In [16]:
%%time
concat_df['가격범위'] = concat_df['거래/보증금(만원)'].progress_apply(price_range)

100%|████████████████████████████████████████████████████████████████████| 5188101/5188101 [00:07<00:00, 660757.73it/s]


Wall time: 7.9 s


In [17]:
def keyword_filter(x):
    # x is a string
    
    if pd.isna(x) == True:
        return x
    
    x = str(x)
    
    special_chars = '!@#$%^&*()[]{};:,./<>?\|`~-=_+차'
    numbers = '0123456789'
    real_estate_type_names = ['아파트', '빌라']
    
    for c in special_chars:
        x = x.replace(c, '')
    
    for n in numbers:
        x = x.replace(n, '')
    
    for name in real_estate_type_names:
        x = x.replace(name, '')
        
    return x

In [18]:
%%time
concat_df['건물단지명_키워드'] = concat_df['건물/단지명'].apply(keyword_filter)

Wall time: 23.9 s


In [19]:
def famous_brand(x):
    # x is a string
    
    if pd.isna(x) == True:
        return x
    
    x = str(x)
    
    famous_brands_list = ['래미안', '레미안', '포레나', '힐스테이트', '더샵', '푸르지오', '롯데캐슬', '아이파크', '편한세상',
                          '디에르트', '우미 린', '디에이치', '르엘', '아크로', '아펠바움', '호반', '써밋', '데시앙', '대방', '코오롱',
                          '중흥', '리슈', '한신', '삼성', '현대', 'LG', 'SK'
                         ]
    
    for brand in famous_brands_list:
        if brand in x:
            return brand
    
    return '기타'

In [21]:
%%time
concat_df['브랜드'] = concat_df['건물/단지명'].progress_apply(famous_brand)

100%|████████████████████████████████████████████████████████████████████| 5188101/5188101 [00:09<00:00, 565211.78it/s]

Wall time: 9.2 s


In [22]:
%%time
concat_df.to_csv('./Prepped Data/main_data_baseline_20211202.csv', index=False)
print(concat_df.shape)
concat_df.head()

(5188101, 18)
Wall time: 24.8 s


,시군구,번지,본번,부번,건물/단지명,거래유형,전용면적(㎡),계약년월,계약일,거래/보증금(만원),월세(만원),층,건축년도,도로명,부동산유형,가격범위,건물단지명_키워드,브랜드
0,서울특별시 강남구 개포동,655-2,655.0,2.0,개포2차현대아파트(220),전세,77.75,201101,5,35000,0,7.0,1988.0,언주로 103,아파트,34000~36000,개포현대,현대
1,서울특별시 강남구 개포동,655-2,655.0,2.0,개포2차현대아파트(220),전세,77.75,201101,18,20000,0,8.0,1988.0,언주로 103,아파트,18000~20000,개포현대,현대
2,서울특별시 강남구 개포동,655-2,655.0,2.0,개포2차현대아파트(220),전세,77.75,201102,1,24000,0,5.0,1988.0,언주로 103,아파트,22000~24000,개포현대,현대
3,서울특별시 강남구 개포동,655-2,655.0,2.0,개포2차현대아파트(220),전세,77.75,201102,11,31000,0,9.0,1988.0,언주로 103,아파트,30000~32000,개포현대,현대
4,서울특별시 강남구 개포동,655-2,655.0,2.0,개포2차현대아파트(220),전세,77.75,201102,24,30500,0,9.0,1988.0,언주로 103,아파트,30000~32000,개포현대,현대
